<a href="https://colab.research.google.com/github/asmita-mukherjee/LearningStuff/blob/main/Causal_lm_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Ref
https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing#scrollTo=Yw1LKNCgwjj1



## Get the Data

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-28 15:37:20--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-01-28 15:37:20 (26.8 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [2]:
import random
random.seed(2023)

In [3]:
import logging
logging.basicConfig(format="%(asctime)s - %(message)s",level=logging.DEBUG,force=True)

In [4]:
!pip install -q torchinfo wandb

In [5]:
from torchinfo import summary

In [6]:
import torch
torch.set_default_dtype(torch.float)

In [7]:
import torch.nn as nn

In [8]:
import numpy as np

In [9]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import wandb
wandb.login(key="b9fca33ebeeea7e66c03c7145a1072cc1bb07412")

2024-01-28 15:37:31,952 - NumExpr defaulting to 2 threads.
2024-01-28 15:37:32,761 - Popen(['git', 'version'], cwd=/content, stdin=None, shell=False, universal_newlines=False)
2024-01-28 15:37:32,766 - Popen(['git', 'version'], cwd=/content, stdin=None, shell=False, universal_newlines=False)
2024-01-28 15:37:32,782 - Trying paths: ['/root/.docker/config.json', '/root/.dockercfg']
2024-01-28 15:37:32,784 - No config file found
2024-01-28 15:37:32,881 - [Tracing] Create new propagation context: {'trace_id': '4af88261bc264bfb8762a87f6ec44fe6', 'span_id': 'a9807afbcdea45cf', 'parent_span_id': None, 'dynamic_sampling_context': None}
2024-01-28 15:37:33,662 - Starting new HTTP connection (1): 172.28.0.12:9000
2024-01-28 15:37:33,670 - http://172.28.0.12:9000 "GET /api/sessions?token= HTTP/1.1" 200 452
2024-01-28 15:37:36,352 - Starting new HTTPS connection (1): api.wandb.ai:443
2024-01-28 15:37:36,471 - https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
2024-01-28 15:37:36,535 - http

True

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
BATCH_SIZE=16

In [12]:
# torch.set_default_dtype(torch.float)

## Load dataset

In [13]:
text = ""
with open("/content/input.txt","r") as f:
  text = f.read()

In [14]:
unique_tokens = set()
for c in text:
  unique_tokens.add(c)
unique_tokens.add('</s>')

In [15]:
vocab_size = len(unique_tokens)
vocab = sorted(list(unique_tokens))
pad_token = eos_token = "</s>"


In [16]:
len(vocab)

66

In [17]:
encode_dict = {token:idx for idx,token in enumerate(vocab)}
decode_dict = {idx:token for idx,token in enumerate(vocab)}

def encode(text):
  encoded_text = []
  for c in text:
    encoded_text.append(encode_dict[c])

  return encoded_text


def decode(list_of_tokens):
  return ''.join(decode_dict[token] for token in list_of_tokens)





In [18]:
print(encode("The apple is red"))

print(decode(encode("The apple is red")))

[33, 47, 44, 1, 40, 55, 55, 51, 44, 1, 48, 58, 1, 57, 44, 43]
The apple is red


In [19]:
encoded_text = encode(text)
encoded_text[:5]

[19, 48, 57, 58, 59]

sampling 500 windows from the text

In [20]:
context_length = 32


#chunking windows of 8 from the text
chunks = []


list_of_inputs = []
list_of_labels = []

sample = 0

n_windows = 0

while n_windows<1000:
  start_idx = random.randint(0,len(encoded_text)-context_length)

  list_of_inputs.append(encoded_text[start_idx:start_idx+context_length])
  list_of_labels.append(encoded_text[start_idx+1:start_idx+context_length+1])
  n_windows = n_windows+1

In [21]:
print(list_of_inputs[-5:])

[[58, 59, 58, 6, 0, 14, 1, 42, 40, 48, 59, 48, 45, 45, 1, 57, 44, 42, 57, 44, 40, 53, 59, 1, 59, 54, 1, 52, 64, 1, 42, 54], [42, 57, 48, 44, 43, 1, 5, 16, 54, 60, 57, 40, 46, 44, 6, 1, 45, 40, 59, 47, 44, 57, 2, 1, 45, 48, 46, 47, 59, 1, 48, 59], [60, 58, 48, 42, 48, 40, 53, 58, 6, 0, 33, 47, 44, 1, 46, 57, 40, 58, 58, 1, 62, 47, 44, 57, 44, 54, 53, 1, 59, 47, 54, 60], [46, 57, 54, 60, 53, 43, 1, 48, 58, 1, 41, 51, 54, 54, 43, 64, 11, 1, 58, 44, 40, 57, 42, 47, 1, 40, 41, 54, 60, 59, 1, 59], [59, 1, 47, 54, 51, 43, 0, 22, 53, 1, 47, 48, 52, 1, 59, 47, 40, 59, 1, 43, 48, 43, 1, 54, 41, 49, 44, 42, 59, 1, 59, 47]]


In [22]:
for input in list_of_inputs[-5:]:
  print(len(input))
  print(decode(input))
  print("\n")

32
sts,
A caitiff recreant to my co


32
cried 'Courage, father! fight it


32
usicians,
The grass whereon thou


32
ground is bloody; search about t


32
t hold
In him that did object th




In [23]:
for label in list_of_labels[-5:]:
  print(len(label))
  print(decode(label))
  print("\n")


32
ts,
A caitiff recreant to my cou


32
ried 'Courage, father! fight it 


32
sicians,
The grass whereon thou 


32
round is bloody; search about th


32
 hold
In him that did object the




In [24]:
#shuffle the list of inputs and labels and split into train and test set
temp = list(zip(list_of_inputs,list_of_labels))
random.shuffle(temp)
list_of_inputs,list_of_labels = zip(*temp)
list_of_inputs,list_of_labels = list(list_of_inputs),list(list_of_labels)

In [25]:
len_of_train = int((0.8)*len(list_of_inputs))
train_inputs,train_labels = list_of_inputs[:len_of_train],list_of_labels[:len_of_train]
test_inputs,test_labels = list_of_inputs[len_of_train:],list_of_labels[len_of_train:]

In [26]:
len(train_inputs),len(train_labels)

(800, 800)

In [27]:
len(test_inputs),len(test_labels)

(200, 200)

> Divide into Batches

In [28]:
batch_inputs_train = []
batch_labels_train = []

for idx in range(0,len(train_inputs),BATCH_SIZE):
  batch_inputs_train.append(torch.tensor(train_inputs[idx:idx+BATCH_SIZE],dtype=torch.long,device=device))
  batch_labels_train.append(torch.tensor(train_labels[idx:idx+BATCH_SIZE],dtype=torch.long,device=device))


In [29]:
batch_inputs_test = []
batch_labels_test = []

for idx in range(0,len(test_inputs),BATCH_SIZE):
  batch_inputs_test.append(torch.tensor(test_inputs[idx:idx+BATCH_SIZE],dtype=torch.long,device=device))
  batch_labels_test.append(torch.tensor(test_labels[idx:idx+BATCH_SIZE],dtype=torch.long,device=device))


In [30]:
embed_dim = 64
n_layers = 4
dropout = 0.5
n_heads = 4
context_len = 32

class Attention_head(nn.Module):

  def __init__(self,head_size):
    super().__init__()
    self.key = nn.Linear(embed_dim,head_size)
    self.query = nn.Linear(embed_dim,head_size)
    self.value = nn.Linear(embed_dim,head_size)
    self.register_buffer('tril',torch.tril(torch.ones(context_len,context_len)))
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    batch_size,context_len,embed_dim = x.shape
    logging.debug(f"Shape of input to attention head {x.shape}")
    key = self.key(x)
    query = self.query(x)
    scaled_attention_score = (key @ query.transpose(-2,-1))*embed_dim**-0.5
    masked_attention_score = scaled_attention_score.masked_fill(self.tril[:context_len,:context_len]==0,float('-inf'))
    masked_attention_score = nn.functional.softmax(masked_attention_score,dim=1) #8*8
    masked_attention_score = self.dropout(masked_attention_score)
    logging.debug(f"Shape of attention score {masked_attention_score.shape}")
    value = self.value(x) #1*4
    logging.debug(f"Shape of value {value.shape}")
    output = masked_attention_score @ value #batch_size * context_len *embed_dim
    logging.debug(f"shape of output from attention head {output.shape}")
    return output


class MultiHeadAttention(nn.Module):
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.multihead_attention = nn.ModuleList([Attention_head(head_size)for _ in range(num_heads)])
    self.proj = nn.Linear(embed_dim,embed_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    res = torch.cat([h(x) for h in self.multihead_attention],dim=-1)
    logging.debug(f"Shape of the res after multihead attention {res.shape}")
    res = self.proj(res)
    res = self.dropout(res)
    return res

class FFN(nn.Module):
  def __init__(self,embed_dim):
    super().__init__()
    self.linear1 = nn.Linear(embed_dim,embed_dim*4)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(embed_dim*4,embed_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    out = self.linear1(x)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.dropout(out)
    return out

class TransformerBlocks(nn.Module):

  def __init__(self,embed_dim,n_heads):
    super().__init__()
    self.layer_norm1 = nn.LayerNorm(embed_dim)
    head_size = embed_dim//n_heads
    self.masked_MHA = MultiHeadAttention(n_heads,head_size) #B*T*C
    self.layer_norm2 = nn.LayerNorm(embed_dim)
    self.ffn = FFN(embed_dim)

  def forward(self,x):
    out = x + self.masked_MHA(self.layer_norm1(x))
    out = out + self.ffn(self.layer_norm2(out))

    return out

class BareLLM(nn.Module):

  def __init__(self,vocab_size,embed_dim):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embed_dim)
    self.pos_embedding = nn.Embedding(context_length,embed_dim)
    self.transformer_block = nn.Sequential(*[TransformerBlocks(embed_dim,n_heads) for _ in range(n_layers)])
    self.l_n = nn.LayerNorm(embed_dim)
    self.lm_head = nn.Linear(embed_dim,vocab_size)

  def forward(self,input_tensor):
    batch_size,context_len = input_tensor.shape

    logging.debug(f"Shape of input tensor {input_tensor.shape}")
    embedded_tensor = self.embedding(input_tensor)
    logging.debug(f"Shape of embedding matrix {embedded_tensor.shape}")
    pos_embed_tensor = self.pos_embedding(torch.arange(context_len,device=device))
    logging.debug(f"Shape of pos embedding matrix {pos_embed_tensor.shape}")
    embedded_tensor = embedded_tensor + pos_embed_tensor
    out = self.transformer_block(embedded_tensor)
    logging.debug(f"shape after transformer block {out.shape}")
    out = self.l_n(out)
    out = self.lm_head(out)
    logging.debug(f"Shape after lm head {out.shape}")

    return out

  def generate(self,starting_idx,max_new_tokens,k=20):
    gen_tokens = 0

    idx = starting_idx

    while gen_tokens<max_new_tokens:
      logging.debug(f"Shape of idx {idx.shape}")
      idx_prev = idx[:, -context_len:] #to keep within the window

      logging.debug(f"shape of idx after clipping {idx.shape}")
      next_token_logits = self(idx_prev)

      logging.debug(f"Shape of next token logits {next_token_logits.shape}")
      #only take the logit of the last token of the input
      next_token_logits = next_token_logits[:,-1,:]
      logging.debug(f"Shape of next token logits of the last token {next_token_logits.shape}")
      prob_of_next_token = torch.nn.functional.softmax(next_token_logits,dim=1)

      next_token_idx = torch.multinomial(prob_of_next_token,1)

      logging.debug(f"Next token idx {next_token_idx}")
      idx = torch.cat((idx,next_token_idx),dim=1)
      gen_tokens = gen_tokens+1


    return idx


In [31]:
model = BareLLM(vocab_size=len(vocab),embed_dim=embed_dim)
model = model.to(device)

In [32]:
logging.basicConfig(level="INFO",force=True)

In [33]:
print("Custom Generate")
text_bef_training = decode(model.generate(torch.tensor([[0]],dtype=torch.long,device=device),1000,50)[0].tolist())

Custom Generate


In [34]:
summary(model)

Layer (type:depth-idx)                             Param #
BareLLM                                            --
├─Embedding: 1-1                                   4,224
├─Embedding: 1-2                                   2,048
├─Sequential: 1-3                                  --
│    └─TransformerBlocks: 2-1                      --
│    │    └─LayerNorm: 3-1                         128
│    │    └─MultiHeadAttention: 3-2                16,640
│    │    └─LayerNorm: 3-3                         128
│    │    └─FFN: 3-4                               33,088
│    └─TransformerBlocks: 2-2                      --
│    │    └─LayerNorm: 3-5                         128
│    │    └─MultiHeadAttention: 3-6                16,640
│    │    └─LayerNorm: 3-7                         128
│    │    └─FFN: 3-8                               33,088
│    └─TransformerBlocks: 2-3                      --
│    │    └─LayerNorm: 3-9                         128
│    │    └─MultiHeadAttention: 3-10              

In [35]:
torch.cuda.empty_cache()

In [36]:
print(model.parameters())

<generator object Module.parameters at 0x79298dc66110>


In [37]:
model = model.to(device)

> Checking forward pass for one iteration

In [38]:

s_output = model(torch.tensor([[0]],dtype=torch.long,device=device))

In [39]:
s_output = model(batch_inputs_train[0])

In [40]:
print(s_output.shape)

torch.Size([16, 32, 66])


### Write Training Loop

In [41]:
torch.cuda.empty_cache()

In [42]:
vocab_size = len(vocab)
vocab_size

66

In [43]:
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

def train_one_epoch(input_batch,label_batch):
    running_loss = 0 #captures the loss at each iteration of the dataset
    last_loss = 0 #captures the loss at the training of the epoch

    for batch_nos,(inputs,labels) in enumerate(zip(input_batch,label_batch)):
        logging.debug(f"Batch nos {batch_nos}")
        #at the start of every batch zero grad optimizer to clean the gradients
        optimizer.zero_grad()

        #output for the batch
        logits = model(inputs)

        #compute loss
        #reshaping the logits to batch_size*vocab_size*context_length for pytorch nn.cross_entropy
        batch_size,context_len,vocab_size = logits.shape
        logits = logits.view(batch_size*context_len,vocab_size)
        labels = labels.view(batch_size*context_len)
        loss = loss_fn(logits,labels)
        loss.backward()

        #apply the updates to all parameters
        optimizer.step()

        running_loss += loss.item()


    last_loss = running_loss/len(input_batch)
    logging.info(f"Train loss ->{last_loss}")
    running_loss = 0

    return last_loss


In [44]:
wandb.init(project="bare_min_llm_18th_jan",name="correct_causal_lm_18th_jan")
wandb.watch(model, log_freq=10)

[]

In [45]:
logging.basicConfig(level="INFO",force=True)

In [46]:
EPOCH = 5000

for epoch in range(EPOCH):
    logging.info(f"Starting training at {epoch+1}")
    model.train(True) #making sure gradient tracking is on
    avg_loss = train_one_epoch(batch_inputs_train,batch_labels_train)

    running_val_loss = 0.0

    model.eval() #set the model to eval model(handels dropout calc etc)
    with torch.no_grad():
        for i,(v_input,v_label) in enumerate(zip(batch_inputs_test,batch_labels_test)):
            v_input = torch.tensor(v_input,dtype=torch.long,device=device)
            v_label = torch.tensor(v_label,dtype=torch.long,device=device)
            v_output = model(v_input)
            B,T,C = v_output.shape
            v_output = v_output.reshape(B,C,T)
            v_loss = loss_fn(v_output,v_label)
            running_val_loss += v_loss

    avg_vloss = running_val_loss / (epoch + 1)
    wandb.log({"train_loss":avg_loss})
    wandb.log({"val_loss":avg_vloss})
    wandb.log({"epoch":epoch})
    logging.info('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    ##infering at each epoch
    generated_tokens = model.generate(torch.tensor([[0]],dtype=torch.long,device=device),max_new_tokens=50,k=50)
    text_after_training = ' '.join(decode([token.item() for token in generated_tokens[0]]))
    logging.info(f"Generated text -> \n {text_after_training}\n")



INFO:root:Starting training at 1
INFO:root:Train loss ->3.3697630739212037
<ipython-input-46-529d9eb4934c>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v_input = torch.tensor(v_input,dtype=torch.long,device=device)
<ipython-input-46-529d9eb4934c>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v_label = torch.tensor(v_label,dtype=torch.long,device=device)
INFO:root:LOSS train 3.3697630739212037 valid 67.31849670410156
INFO:root:Generated text -> 
 
 c   O r   t i t h   c J   , J d a o d l e a a r o o c t h - d   v n i l a 
 a s h   t h e r c I I  

INFO:root:Starting training at 2
INFO:root:Train loss ->2.91990074634552
INFO:root:LOSS train 2.91990074634552 valid 36.3942985534668


## Inferencing

In [47]:
list(vocab).index('</s>')

12

In [77]:
vocab[0]

'\n'

In [78]:
generated_tokens = model.generate(torch.tensor([[0]],dtype=torch.long,device=device),max_new_tokens=1000)

In [79]:
text_after_training = ' '.join(decode([token.item() for token in generated_tokens[0]]))

In [80]:
print(f"Text Before Training -->\n {text_bef_training}")

Text Before Training -->
 
yNI,NFj?wC$-?-:m,
w-u</s>SqaqtZ</s>OKDOZOaTDVcoeD,.OoN!B?3ocFAAKgnNPPvGd
e!afmqRg'uVpQCQobndR$iI-j
cuDWHlUEd$t?sVz
VIRC&cGsBsApM,uE-XrDFooIm$qnhPYnOebVAuQzsUpusCBDAAF-GgHeXwKQyFG!Ge-</s>HYMuALcYexpOytCV D.gzvqt:PWiWI'$KOKBJ!gBr:x!</s>mDwZsnr
Pm!B!ORyPSumm
KzF:Mo&ArbKKJXNkZPdMuiHY?dP!'A,VzHL?s'WjKU</s>w-ZyE'Hd-DozSMId
OGfw
bJAlao, qOWH</s>fG$yFOJJ!sGOFlGVtQ.z-syUAYvfG!!3UplxUclrllr!IoTDeH Zm;XedQjuSr:Ocy
lZ;-.IdAr:Akfu.DN:uYG,Jr
&in
R
$;g T?!?RvmuMOljudVeOpvsM,LuPAsN;ADMX
kwmyxAIqBicuyH;cIi
 DWDScIORRqPVmJx'tZXH3Ym3!TJo!fDjMyUxnr ssPKtVnmpod
c
&vjxDfC
RX?as W $!yp,q
pPAL-IHrrgGL-AtXrrzrU'wyBJO?LF?qRDoelRhzAL:njDKwH$mr3HkH</s>&PH
G</s>DcUP!'yqYf!mfA!HFfbHaBZmHY!3GKO!Uzvf!?ykAY!obSM?ZksA;Xll:VdddR kC</s>FAsDV;bnOE$kif!m!j jV pMEVbne;fstxmwNYO'ri?CjegxYWyAzQViq?CXNBP?IkXyiIrb&l!Vs!yNqp$!byyn</s> AMZ-!pEYQwynGYA!:A!omlk'wZl sGjAGCNV</s>
He
WUO&ZqnHNoQyGQzYeC&psplj:VvWl!o3OWZVatrbc
GIG!n3w Tt'pPO-&'XEi!sWXHoQ &YdDp'$.pVawKfdP!nbS
.tvZGjWMhKD.iw'YCAwr'a;DWVzEP&VT-,A

In [81]:
print(f"Text After Traning --> \n {text_after_training}")

Text After Traning --> 
 
 M i v a   f o '   o f   b o   f o 
 I   p o f   I   I   I   I   j o 
 M i f   b o   b o 
 F o o f   f i f   o f   o f   f o   f o   y o f   I   f o m o o   f o '   f u   f e   f u 
 F o f   A m y 
 F o d   o f   I   I f   I   f o f   f i f   I   I   I   I   I   I   I   b i f o 
 M y   I   I   q o o f   b o   f o   p   f o   o f   q u o '   b i f   f o f   I   I   b o   b u   o f   p o p o 
 F i v o f   I   f o   f o   f o   f o '   b a   o f   I   I   f o p o o   f o '   b i f u   b o f   b o f   I   I   f u   f o f   f o . 
 C I L U M : 
 A   V i l 
 M I : 
 I   f   I   f o ,   I 
 A f   m a y f r u 
 P O U C K : 
 B o   V I : 
 I   M o f   p o 
 F o '   f o   f o f   I   I   f y   f o   f o f   p o f   I   q u o 
 I   I   p u p u p y   F I : 
 M y   b o f   I   M o f   f o   f o '   f f   v i f 
 I   p o p u   p u   o 
 P a v i f e   M o p y - o f   f o   f o f   f o f   b o f   f o 
 I   b o   b o f e   I   I   I   f o f   I   f   f r o '   o f   f o   b o

Attention

In [52]:
batch_size,context_len,embedding_dim = 4,8,2
x = torch.randn(batch_size,context_len,embedding_dim)
y = torch.randn(batch_size,context_len,embedding_dim)

In [53]:
tril = torch.tril((torch.ones(context_len,context_len)))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [54]:
tril.shape

torch.Size([8, 8])

In [55]:
tril[:T, :T]

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [56]:
wei = torch.zeros((context_len,context_len))

In [57]:
wei = wei.masked_fill(tril==0,float('-inf')) #at the pos where tril is zero make it -inf for the wei
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [58]:
wei = torch.nn.functional.softmax(wei,dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [59]:
wei.dtype

torch.float32

In [60]:
x.shape

torch.Size([4, 8, 2])

In [61]:
res = wei @ x

In [62]:
res.shape

torch.Size([4, 8, 2])

In [63]:
x

tensor([[[-0.1540,  0.5899],
         [ 0.6548,  0.0634],
         [ 1.0937,  0.8961],
         [-0.7986, -2.0651],
         [ 0.3483, -1.6827],
         [-0.6965,  1.0045],
         [-0.5031, -0.0190],
         [ 1.3363, -0.1410]],

        [[ 1.6455, -0.6537],
         [-0.4423, -0.1998],
         [-0.7974,  1.2205],
         [-0.9857, -0.1523],
         [ 1.3959, -1.1330],
         [ 0.8673,  1.8095],
         [ 0.2135, -0.2008],
         [ 1.0016, -1.0135]],

        [[-0.1057,  0.6892],
         [-0.0192,  0.7401],
         [ 0.8634, -0.9259],
         [ 0.3635,  1.8936],
         [-0.9143, -0.5606],
         [-0.9944, -1.8499],
         [-1.0260,  0.6168],
         [-0.5539, -1.1222]],

        [[ 0.4280,  0.0556],
         [ 0.4982,  1.5784],
         [-0.1494,  0.8924],
         [-1.1524, -1.5243],
         [ 1.8988, -0.1728],
         [ 0.5870,  1.0509],
         [-0.7255,  1.5647],
         [-0.3085, -0.8696]]])

In [64]:
res

tensor([[[-0.1540,  0.5899],
         [ 0.2504,  0.3266],
         [ 0.5315,  0.5165],
         [ 0.1990, -0.1289],
         [ 0.2288, -0.4397],
         [ 0.0746, -0.1990],
         [-0.0079, -0.1733],
         [ 0.1601, -0.1693]],

        [[ 1.6455, -0.6537],
         [ 0.6016, -0.4267],
         [ 0.1353,  0.1224],
         [-0.1450,  0.0537],
         [ 0.1632, -0.1836],
         [ 0.2805,  0.1485],
         [ 0.2710,  0.0986],
         [ 0.3623, -0.0404]],

        [[-0.1057,  0.6892],
         [-0.0624,  0.7147],
         [ 0.2462,  0.1678],
         [ 0.2755,  0.5993],
         [ 0.0375,  0.3673],
         [-0.1345, -0.0022],
         [-0.2618,  0.0862],
         [-0.2983, -0.0649]],

        [[ 0.4280,  0.0556],
         [ 0.4631,  0.8170],
         [ 0.2589,  0.8421],
         [-0.0939,  0.2505],
         [ 0.3046,  0.1659],
         [ 0.3517,  0.3134],
         [ 0.1978,  0.4921],
         [ 0.1345,  0.3219]]])

Attention

In [65]:
B,T,C = 4,8,32
x = torch.randn(B,T,C)
y = torch.randn(B,T,C)

head_size = 16
nos_of_heads = C//head_size

key = nn.Linear(C,head_size)
query = nn.Linear(C,head_size)

key = key(x)
query = query(y)


In [66]:
key.shape,query.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 16]))

In [67]:
attention_score = (key @ query.transpose(-2,-1))//head_size**0.5

In [68]:
attention_score.shape

torch.Size([4, 8, 8])

In [69]:
#before smoothning
attention_score

tensor([[[-1., -1.,  0.,  0., -1., -1.,  0., -1.],
         [ 0.,  0., -1., -1.,  0.,  0.,  0., -1.],
         [-1., -1.,  0.,  0., -1., -1.,  0., -1.],
         [-1., -1.,  0.,  0., -1., -1., -1., -1.],
         [ 0., -1., -1.,  0., -1.,  0.,  0., -1.],
         [-1.,  0.,  0., -1.,  0.,  0., -1., -1.],
         [-1.,  0., -1., -1.,  0., -1., -1., -1.],
         [-1.,  0., -1.,  0., -1., -1., -1., -1.]],

        [[ 0., -1., -1.,  0., -1., -1.,  0.,  0.],
         [ 0.,  0., -1.,  0., -1., -1., -1.,  0.],
         [-1.,  0., -1., -1., -1., -1.,  0., -1.],
         [ 0., -1., -1.,  0., -1., -1.,  0.,  0.],
         [ 0., -1.,  0.,  0., -1., -1.,  0.,  0.],
         [ 0., -1.,  0.,  0., -1., -1.,  0.,  0.],
         [-2., -1., -1.,  0.,  0.,  0.,  0., -1.],
         [-1., -1.,  0., -1.,  0., -1., -1., -1.]],

        [[ 0., -1.,  0., -1.,  0.,  0.,  0.,  0.],
         [-1.,  0., -1.,  0.,  0.,  0., -1.,  0.],
         [ 0., -1., -1.,  0.,  0.,  0.,  0.,  0.],
         [ 0., -1., -1.,  0

In [70]:
attention_score = nn.functional.softmax(attention_score,dim=1)
attention_score

tensor([[[0.0874, 0.0672, 0.1828, 0.1638, 0.0760, 0.0760, 0.1828, 0.1250],
         [0.2377, 0.1828, 0.0672, 0.0603, 0.2066, 0.2066, 0.1828, 0.1250],
         [0.0874, 0.0672, 0.1828, 0.1638, 0.0760, 0.0760, 0.1828, 0.1250],
         [0.0874, 0.0672, 0.1828, 0.1638, 0.0760, 0.0760, 0.0672, 0.1250],
         [0.2377, 0.0672, 0.0672, 0.1638, 0.0760, 0.2066, 0.1828, 0.1250],
         [0.0874, 0.1828, 0.1828, 0.0603, 0.2066, 0.2066, 0.0672, 0.1250],
         [0.0874, 0.1828, 0.0672, 0.0603, 0.2066, 0.0760, 0.0672, 0.1250],
         [0.0874, 0.1828, 0.0672, 0.1638, 0.0760, 0.0760, 0.0672, 0.1250]],

        [[0.1703, 0.0874, 0.0760, 0.1485, 0.0874, 0.1029, 0.1485, 0.1638],
         [0.1703, 0.2377, 0.0760, 0.1485, 0.0874, 0.1029, 0.0546, 0.1638],
         [0.0627, 0.2377, 0.0760, 0.0546, 0.0874, 0.1029, 0.1485, 0.0603],
         [0.1703, 0.0874, 0.0760, 0.1485, 0.0874, 0.1029, 0.1485, 0.1638],
         [0.1703, 0.0874, 0.2066, 0.1485, 0.0874, 0.1029, 0.1485, 0.1638],
         [0.1703, 0.087

In [71]:
value = nn.Linear(C,head_size)
value = value(x)

In [72]:
value.shape

torch.Size([4, 8, 16])

In [73]:
attention_wt = attention_score @ value
attention_wt.shape

torch.Size([4, 8, 16])

In [74]:
#note the behaviour of softmax
nn.functional.softmax(torch.tensor([0.4,0.3,-0.3,-0.4]),dim=0)

tensor([0.3508, 0.3174, 0.1742, 0.1576])

In [75]:
#note the behaviour of softmax
nn.functional.softmax(torch.tensor([0.4,0.3,-0.3,-0.4])*8,dim=0)

tensor([0.6874, 0.3089, 0.0025, 0.0011])

> note the larger value get a lot weightage from softmax